We solve the steady advection-diffusion equation $$-\kappa \Delta u + \mathbf{a}\cdot \nabla u = 0, \quad\text{where }\kappa =10^{-7} \text{ and } \mathbf{a} = \left(-y + 0.5\,,\,\, x - 0.5\right)^T $$

The domain is $0\le x, y\le 1$ and our solution satisfies the dirichlet boundary conditions indicated below. Especially important is the sine wave that extends from the center of the square to an edge.
<img src="files/ProblemSetup.png">
(pictures taken from this [paper](https://link.springer.com/content/pdf/10.1007/s11831-010-9042-5.pdf))

The exact solution (in the hyperbolic limit) is a solid of revolution of the sine wave about the z-axis through the center of the square.

In this notebook, we will solve the problem using a multipatch topology. We divide our unit square into four patches. 



## Galerkin

**Weak Form**: $$0 = \int_\Omega \kappa \, N_{A, j}\, u_{, j} \,-\, u\, N_{A, k} \,a_k \, dx$$
where $N_i$ is a basis function.

In [19]:
import nutils as ntl
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import collections
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook

degree = 1 #degree of basis functions
nelems = 10 #number of elements along edge of unit square


#Connectivity :  6──7──9
#                │  │  │
#                3──4──5
#                │  │  │
#                0──1──2

topo, geom =ntl.mesh.multipatch(patches=[[0, 3, 1, 4], [3, 6, 4, 7], [4, 7, 5, 8], [1, 4, 2, 5]],
                                patchverts=[[0,0], [.5, 0], [1, 0], [0, .5], [.5, .5], [1, .5], [0, 1], [.5, 1], [1, 1]],
                               nelems= nelems)

print(type(topo))

ns = ntl.function.Namespace()
ns.x = geom
ns.a = [- geom[1] + 0.5, geom[0] - 0.5]  
ns.kappa = 1e-7
ns.pi = np.pi
ns.basis = topo.basis('std', degree = degree)
ns.u = 'basis_i ?dofs_i'
 
print(topo ) 

#Dirichlet boundary condition
#tophalf of domain is defined via y - 0.5 > 0. Midline is one of the boundaries of this new domain 
#halftopo = topo.trim(geom[1] - 0.50, maxrefine = 0, name= 'halfbnd') 
#halfbnd = halftopo.boundary['halfbnd']
#midline = halfbnd.trim(geom[0] - 0.5, maxrefine = 0) #Pick variable names better than I do
#madline = midline.trim(0.75 - geom[1], maxrefine = 0) # this is the boundary we want

#print(type(halfbnd), type(midline), type(madline))

#sqr = madline.integral('( u - sin(2 pi (x_0 - 0.5))  )^2 d:x' @ ns, degree = degree * 2)#

sqr += topo.boundary['left'].integral( '(u - 0)^2 d:x' @ ns, degree = degree*2 )
sqr += topo.boundary['bottom'].integral('(u - 0)^2 d:x' @ ns, degree = degree*2)
sqr += topo.boundary['right'].integral( '(u - 0)^2 d:x' @ ns, degree = degree*2 )
sqr += topo.boundary['top'].integral( '(u - 1)^2 d:x' @ ns, degree = degree*2 )
cons = ntl.solver.optimize('dofs', sqr, droptol=1e-15) #this applies the boundary condition to u

#residual
res = topo.integral( '(kappa basis_i,j u_,j  - u basis_i,n a_n) d:x' @ ns, degree = degree * 2) 
lhs = ntl.solver.solve_linear('dofs', res, constrain= cons)

#this converts our solution into arrays
bezier = topo.sample('bezier', 2)
x = bezier.eval('x_i' @ ns)
u = bezier.eval('u' @ ns, dofs = lhs)

#ntl.export.triplot('RF_Gal.png', x, u, tri=bezier.tri, hull=bezier.hull)
fig = plt.figure(figsize = (5,5))
ax = fig.add_subplot(111, aspect= 'equal')
im = ax.tripcolor(x[:,0], x[:, 1], bezier.tri, u, cmap = plt.cm.Spectral)
fig.colorbar(im)
ax.autoscale(enable= True, axis = 'both', tight = 'True')
ax.set_title('Galerkin solution')
ax.add_collection(collections.LineCollection(x[bezier.hull], colors='k', linewidths= .1))
plt.axis('off')

fig2 = plt.figure(figsize= (5,5))
ax2 = fig2.add_subplot(111, projection ='3d')
ax2.plot_trisurf( x[:, 0], x[:,1], u, cmap=plt.cm.Spectral)

<class 'nutils.topology.MultipatchTopology'>
MultipatchTopology(#400)


KeyError: ('left',)

In [9]:
help(ntl.topology.MultipatchTopology)

Help on class MultipatchTopology in module nutils.topology:

class MultipatchTopology(Topology)
 |  MultipatchTopology(patches)
 |  
 |  multipatch topology
 |  
 |  Method resolution order:
 |      MultipatchTopology
 |      Topology
 |      nutils.types.Singleton
 |      nutils.types.Immutable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, patches)
 |      constructor
 |  
 |  basis_patch(self)
 |      degree zero patchwise discontinuous basis
 |  
 |  basis_spline(self, degree, patchcontinuous=True, knotvalues=None, knotmultiplicities=None, *, continuity=-1)
 |      spline from vertices
 |      
 |      Create a spline basis with degree ``degree`` per patch.  If
 |      ``patchcontinuous``` is true the basis is $C^0$-continuous at patch
 |      interfaces.
 |  
 |  getitem(self, key)
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  build_boundarydata(connectivity)
 |      build b